In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

### The goal of this notebook is to see how effective our pybmc packages are doing

In [2]:
import importlib
import dataset_trial2
import bmc
importlib.reload(dataset_trial2)
importlib.reload(bmc)

from dataset_trial2 import Dataset
from bmc import BayesianModelCombination

The number of common isotopes are(629, 2)isotopes
The number of selected stable isotopes are(167, 2)isotopes
       N    Z    ME2  MEdelta    PC1   NL3S      SKMS       SKP      SLY4  \
0      8    8  2.729    2.691  2.747  2.735  2.794203  2.824011  2.795792   
1     10    8  2.733    2.704  2.749  2.728  2.793734  2.825080  2.799074   
2     12    8  2.743    2.720  2.757  2.730  2.797764  2.832105  2.807422   
3     14    8  2.757    2.739  2.770  2.739  2.807942  2.843493  2.820317   
4     16    8  2.778    2.764  2.788  2.759  2.828637  2.864051  2.841335   
..   ...  ...    ...      ...    ...    ...       ...       ...       ...   
624  154  106  6.097    6.083  6.093  6.099  6.083176  6.094002  6.085548   
625  156  106  6.109    6.097  6.106  6.111  6.092065  6.102800  6.094902   
626  156  108  6.123    6.117  6.122  6.126  6.109944  6.126930  6.115375   
627  158  108  6.136    6.128  6.134  6.138  6.119354  6.135442  6.124092   
628  160  110  6.162    6.158  6.155  6.166 

In [3]:
uploaded_data = Dataset(data_source= r'C:\Users\congn\OneDrive\Desktop\An Le Materials\ModelOrthogonalization\data\selected_data.h5',models = None,keys = None)

# We have checked that it does extract common isotopes for you
common_isotopes, common_isotopes_df = uploaded_data.extract_common_isotopes()

# This does give you the subset of data that we will work with
selected_NZ, selected_NZ_df = uploaded_data.get_subset( common_isotopes_df, domain_X='even-even', N_range=[8, 300], Z_range= [8, 300])
print('The number of common isotopes are' + str(selected_NZ_df.shape) + 'isotopes')

# We want to check if the get_subset method could handle more than just the dataset of original models
stable_isotopes_full=np.loadtxt(r"C:\Users\congn\OneDrive\Desktop\An Le Materials\ModelOrthogonalization\Stable-Isotopes.txt")
stable_isotopes_full_df = pd.DataFrame({ 'N' : stable_isotopes_full[:, 0], 'Z' : stable_isotopes_full[:, 1]  })
selected_stable_isotopes, selected_stable_isotopes_df = uploaded_data.get_subset(stable_isotopes_full_df,\
                                                                                  domain_X='even-even', N_range=[8, 300], Z_range= [8, 300])
print('The number of selected stable isotopes are' + str(selected_stable_isotopes.shape) + 'isotopes')

# This check that the code is able to extract dataframe of models 
selected_models_data_sets_radius_df = uploaded_data.selected_models_data_sets_extraction(initial_NZ_df= selected_NZ_df, property= 'ChRad')
selected_models_data_sets_mass_df = uploaded_data.selected_models_data_sets_extraction(initial_NZ_df= selected_NZ_df, property= 'BE')
selected_models_data_sets_radius_df

The number of common isotopes are(629, 2)isotopes
The number of selected stable isotopes are(167, 2)isotopes


,N,Z,ME2,MEdelta,PC1,NL3S,SKMS,SKP,SLY4,SV,UNEDF0,UNEDF1
0,8,8,2.729,2.691,2.747,2.735,2.794203,2.824011,2.795792,2.817382,2.793546,2.790816
1,10,8,2.733,2.704,2.749,2.728,2.793734,2.825080,2.799074,2.807857,2.785015,2.781329
2,12,8,2.743,2.720,2.757,2.730,2.797764,2.832105,2.807422,2.810681,2.786540,2.788547
3,14,8,2.757,2.739,2.770,2.739,2.807942,2.843493,2.820317,2.820795,2.796530,2.802832
4,16,8,2.778,2.764,2.788,2.759,2.828637,2.864051,2.841335,2.841434,2.819096,2.825622
...,...,...,...,...,...,...,...,...,...,...,...,...
624,154,106,6.097,6.083,6.093,6.099,6.083176,6.094002,6.085548,6.070406,6.073800,6.089964
625,156,106,6.109,6.097,6.106,6.111,6.092065,6.102800,6.094902,6.079813,6.083373,6.099768
626,156,108,6.123,6.117,6.122,6.126,6.109944,6.126930,6.115375,6.100274,6.103005,6.118907
627,158,108,6.136,6.128,6.134,6.138,6.119354,6.135442,6.124092,6.109421,6.112236,6.128383


In [4]:
a = pd.read_csv(r"C:\Users\congn\OneDrive\Desktop\An Le Materials\ModelOrthogonalization\data\charge_radii.csv")
a.rename(columns = {'z' : 'Z', 'n' : 'N'}, inplace = True) # Rename the column N, Z

In [5]:
# This dataset does not yet contain the real data, let's now merge this dataset with the real data
truth_BE = pd.read_hdf(r'C:\Users\congn\OneDrive\Desktop\An Le Materials\ModelOrthogonalization\data\selected_data.h5', key = 'AME2020')
truth_BE_df = pd.DataFrame(truth_BE)
selected_models_data_sets_mass = pd.merge(selected_models_data_sets_mass_df, truth_BE[['N', 'Z', 'BE']], on = ['N', 'Z'], how = 'inner') #Add real data to the dataframe
selected_models_data_sets_mass.rename(columns = {'BE' : 'truth'}, inplace = True) #Rename our truth column
cols_mass = list(selected_models_data_sets_mass.keys())
cols_mass.remove('truth')
cols_mass.insert(2, 'truth') # These 2 lines rearrange the order fo the column
selected_models_data_sets_mass = selected_models_data_sets_mass[cols_mass]

# This is for the radius data
truth_CR_df = a
selected_models_data_sets_radius = pd.merge(selected_models_data_sets_radius_df, truth_CR_df[['N', 'Z',  'radius_val']], how = 'inner', on = ['N', 'Z'])
selected_models_data_sets_radius.rename(columns = {'radius_val' : 'truth'}, inplace = True)
cols_radius = list(selected_models_data_sets_radius.keys())
# The next 2 lines help you to transition the truth column to the start of the dataframe
cols_radius.remove('truth')
cols_radius.insert(2, 'truth')
selected_models_data_sets_radius = selected_models_data_sets_radius[cols_radius]
selected_models_data_sets_radius = selected_models_data_sets_radius.copy().dropna().reset_index(drop = True)
selected_models_data_sets_radius

,N,Z,truth,ME2,MEdelta,PC1,NL3S,SKMS,SKP,SLY4,SV,UNEDF0,UNEDF1
0,8,8,2.6991,2.729,2.691,2.747,2.735,2.794203,2.824011,2.795792,2.817382,2.793546,2.790816
1,10,8,2.7726,2.733,2.704,2.749,2.728,2.793734,2.825080,2.799074,2.807857,2.785015,2.781329
2,8,10,2.9714,2.956,2.912,2.945,2.955,2.956188,2.987489,2.964309,2.991813,2.966094,2.973445
3,10,10,3.0055,2.986,2.942,2.975,2.973,2.942506,2.973045,2.951974,2.974810,2.935663,2.967315
4,12,10,2.9525,2.960,2.949,2.978,2.941,2.964141,2.972133,2.976913,2.981346,2.925008,2.968485
...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,150,94,5.8948,5.933,5.920,5.930,5.942,5.910061,5.918577,5.915247,5.902641,5.908618,5.923415
341,146,96,5.8285,5.935,5.921,5.933,5.936,5.919476,5.928108,5.924384,5.908823,5.911968,5.928126
342,148,96,5.8429,5.948,5.938,5.947,5.951,5.929864,5.939063,5.934703,5.920200,5.923912,5.940104
343,150,96,5.8562,5.958,5.945,5.956,5.964,5.939141,5.949234,5.944656,5.930929,5.935161,5.951470
